In [59]:
import os
import tempfile
import urllib3
from dotenv import load_dotenv
import streamlit as st

# 경고 비활성화 (SSL 관련)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from openai import OpenAI
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader, WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import Document

In [60]:
#Chroma tenant 오류 방지 위한 코드
import chromadb
chromadb.api.client.SharedSystemClient.clear_system_cache()

#오픈AI API 키 설정
load_dotenv()  # 현재 경로의 .env 로드
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
if not os.environ['OPENAI_API_KEY']:
    raise ValueError('OPENAI_API_KEY not found in environment. set it in .env or env vars')

## 1. 문서 불러오기

In [61]:
import tempfile

In [62]:
file_path = '../data/langchainThon/cv/경력기술서_천승우.docx'
# 파일을 바이너리 모드로 읽어서 _file처럼 흉내내기
class DummyFile:
    def __init__(self, path):
        self.name = path
        with open(path, "rb") as f:
            self._data = f.read()
    def getvalue(self):
        return self._data

_file = DummyFile(file_path)

In [63]:
def load_cv(_file):
    with tempfile.NamedTemporaryFile(mode='wb', delete=False) as tmp_file:
        tmp_file.write(_file.getvalue())
        tmp_file_path = tmp_file.name
        
    #PDF 파일 업로드
    if _file.name.endswith('.pdf'):
        loader = PyPDFLoader(file_path=tmp_file_path)
    
    #Word 파일 업로드
    elif _file.name.endswith('.docx'):
        loader = Docx2txtLoader(file_path=tmp_file_path)
    
    # 파일 형식이 틀릴경우 에러 메세지 출력
    else:
        raise ValueError("지원하지 않는 파일 형식입니다. PDF 또는 DOCX만 업로드해주세요.")
    
    pages = loader.load()

    return pages

In [64]:
# cv pages
print(load_cv(_file))

[Document(metadata={'source': 'C:\\Users\\user\\AppData\\Local\\Temp\\tmp9b__f75e'}, page_content='경력기술서\n\n[개인 정보]\n\n이메일: fourleaves8@gmail.com\n\n연락처: 010-4788-7980\n\n\n\n[전문 분야]\n\n• 데이터 사이언스       \t\t • AI 엔지니어링\t\n\n• 데이터 분석           \t\t • 프롬프트 엔지니어링\n\n• 생물분자 및 화학공학 연구\n\n\n\n[요약]\n\n저는 생명·의학 데이터를 처리하고 분석 플랫폼을 구축한 경험 및 복잡한 데이터를 효율적으로 분석해 의사결정을 지원하는 역량을 갖추고 있습니다. Python 기반의 데이터 전처리, 알고리즘 고도화, 분류 및 예측 모델 개발을 통해 항암제 신규 적응증 탐색 플랫폼을 성공적으로 구현한 바 있습니다. 또한 데이터 수집·정규화·시각화 전 과정을 주도하며, 스타트업의 코스닥 상장 및 실제 제약사 프로젝트에 기여한 실무 경험을 쌓았습니다. \n\n\n\n[보유 역량 및 기술]\n\n프로그래밍 언어\n\nPython, R, SQL, Java\n\n도구 및 프레임워크\n\npandas, NumPy, Matplotlib, seaborn, Plotly, scikit-learn, PyTorch, TensorFlow, RAG, LangChain, MySQL, Oracle\n\n운영체제\n\nLinux(Ubuntu), Windows, macOS\n\nPalantir Foundry\n\nPipeline Builder, Contour, Ontology(Action), Workshop, AIP, Dataset\n\n기타\n\nVim / Vi, Git / GitHub, Bioinformatics Tools (Gene Set Enrichment Analysis, Bismark, Metilene, lifelines, survival, survminer, lifelines)\n

In [65]:
def load_jd(_url: str):
    loader = WebBaseLoader(_url)
    loader.requests_kwargs = {'verify': False}  # SSL 검증 비활성화
    
    pages = loader.load()
    
    return pages

In [66]:
# jd pages
print(load_jd('https://www.wanted.co.kr/wd/297589'))

[Document(metadata={'source': 'https://www.wanted.co.kr/wd/297589', 'title': '[임팩티브에이아이] [Deepflow] 데이터 사이언티스트 (Data Scientist) 채용 공고 | 원티드', 'description': '임팩티브에이아이의 [Deepflow] 데이터 사이언티스트 (Data Scientist) 포지션을 확인해 보세요. 취업·이직에 성공하면, 합격보상금 50만원을 드립니다.\n포지션: [Deepflow] 데이터 사이언티스트 (Data Scientist)\n회사 위치: 서울 강남구\n자격 요건: • 시계열 예측 모델 및 시스템 구축 프로젝트 경험자\n• 인공지능, 데이터과학, 전산학, 통계 등 관련 전공 석사학위 이상\n• 머신러닝 및 데이터 분석 분야에서 5년 이상 경력 보유하신 분\n주요 업무: 임팩티브AI의 데이터사이언티스트는 머신러닝/딥러닝 모델 구축, 시계열 예측, 데이터분석 업무를 수행하게 됩니다. \n소프트웨어 엔지니어, 서비스 기획자, 도메인 전문가와 협력하여, 정확하고 확장가능한 예측모델을 개발 및 최적화하는 역할을 담당하게 됩니다.\n\n  • 수요예측 위한 데이터 분석 및 예측모델 설계/개발\n  • 데이터 전처리 및 피처 엔지니어링\n  • 인과 추론 및 설명가능성 구현을 위한 분석\n  • 고객사 딜리버리 위한 예측모델 구축 및 배포\n  • 최신 모델 및 데이터 분석 트렌드 연구 및 적용', 'language': 'ko-KR'}, page_content='[임팩티브에이아이] [Deepflow] 데이터 사이언티스트 (Data Scientist) 채용 공고 | 원티드채용이력서교육•이벤트콘텐츠소셜프리랜서더보기회원가입/로그인회원가입기업 서비스1/6임팩티브에이아이∙서울 강남구∙경력 5-12년[Deepflow] 데이터 사이언티스트 (Data Scientist)포지션 상세[임팩티브AI 소개]임팩티브AI는 수요예측에 특화된 AI 솔루션 전문기업입니다. 자체 솔루션 Deepflow

## 2. 텍스트 나누기, 임베딩, 벡터DB 생성

In [67]:
def chunk_documents(pages, source_label, chunk_size=500, chunk_overlap=100):
    '''
    페이지(Document list)들을 청크로 나누고 metadata를 추가하는 함수
    - pages: Document list
    - source_label: 'cv' 또는 'jd' 등 source 표시
    - return: 청크가 나뉜 Document 리스트
    '''
    text_splitter = RecursiveCharacterTextSplitter(
        separators=['\n\n', '\n'],
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        is_separator_regex=False
    )

    split_docs = text_splitter.split_documents(pages)

    split_docs_with_meta = [
        Document(page_content=d.page_content,
                 metadata={'source': source_label, 'chunk_id': str(i)})
        for i, d in enumerate(split_docs)
    ]

    return split_docs_with_meta

In [68]:
chunk_documents(load_jd('https://www.wanted.co.kr/wd/297589'), 'jd')

[Document(metadata={'source': 'jd', 'chunk_id': '0'}, page_content='[임팩티브에이아이] [Deepflow] 데이터 사이언티스트 (Data Scientist) 채용 공고 | 원티드채용이력서교육•이벤트콘텐츠소셜프리랜서더보기회원가입/로그인회원가입기업 서비스1/6임팩티브에이아이∙서울 강남구∙경력 5-12년[Deepflow] 데이터 사이언티스트 (Data Scientist)포지션 상세[임팩티브AI 소개]임팩티브AI는 수요예측에 특화된 AI 솔루션 전문기업입니다. 자체 솔루션 Deepflow 통해 제품 수요예측, 원자재 가격예측, 신제품 성과예측을 수행, 재고관리 및 원자재 구매관리 최적화를 실현합니다.  제조·유통 산업에서 기술력을 인정받으며, 고객사의 성과 창출을 지원하고 있습니다[AI 기반 예측 솔루션, Deepflow]•Deepflow Forecast - 제품의 향후 6~12개월 판매량과 출고량을 예측, 수요예측 기반의 재고관리 최적화•Deepflow Materials - 원자재 가격 변동 예측을 통해 최적의 구매 타이밍 확보•Deepflow NPD - 신제품 기획 단계에서 성과 예측을 통해 잘 팔릴 제품의 프로파일 도출•Deepflow Marketview - LLM 기반 모델로 소비자 페르소나를 분석하고 제품에 대한 선호도 예측[임팩티브AI의 강점]• 검증된 기술력과 특허   - 25년 7월 기준 61건의 AI관련 특허를 출원 및 등록하며 지속적으로 기술적 경쟁력을 확보해오고 있습니다.• 글로벌 확장 중   - 임팩티브AI는 글로벌 시장에서도 빠르게 성장하며 미국, 아프리카, 동남아 등으로 사업을 확장하고 있습니다.       특히 아프리카 지역의 AI교육봉사 등 사회공헌활동도 꾸준히 진행중입니다.[회사 성과]·  시리즈-A 82억원 투자 유치·  2024 과학기술정보통신부 장관상 수상·  2024 벤처창업진흥유공포상 중기부 장관상 수상·  2024년도 전년대비 매출 및 고객사 3.5배 성장·  AI관련 국내

In [69]:
def create_vector_store(_cv, _jd):
    cv_docs = chunk_documents(_cv, 'cv')
    jd_docs = chunk_documents(_jd, 'jd')

    # OpenAIEmbeddings
    embeddings_model = OpenAIEmbeddings(model='text-embedding-3-small')
    
    # 새 vectorstore 생성
    vectorstore = Chroma.from_documents(
        documents=(cv_docs+jd_docs),
        embedding=embeddings_model,
        persist_directory=None
    )
    return vectorstore

In [70]:
vectorstore = create_vector_store(_cv=load_cv(_file), _jd=load_jd('https://www.wanted.co.kr/wd/297589'))

In [71]:
cv_docs_count = len([
    d for d in vectorstore._collection.get(include=['metadatas', 'documents'])['metadatas']
    if d['source'] == 'cv'
])
cv_docs_count

11

In [72]:
jd_docs_count = len([
    d for d in vectorstore._collection.get(include=["metadatas", "documents"])['metadatas']
    if d['source'] == 'jd'
])
jd_docs_count

2

In [73]:
vectorstore._collection.count()

13

## 3. Retriever 구성 (검색 옵션: simple similarity or MMR)

In [45]:
# retriever 설정
retriever = vectorstore.as_retriever(
    search_type="mmr",  # 다양성을 고려한 검색
    search_kwargs={
        "k": 5,       # 최종 반환할 문서 수
        "fetch_k": 20  # 검색 후보로 고려할 청크 수
    }
)

query = "내가 다녔던 회사를 알아맞춰봐"
docs = retriever.invoke(query)

# 청크 내용을 하나로 합쳐서 GPT-40-mini에 전달
context = "\n\n".join([d.page_content for d in docs])

In [46]:
docs

[Document(id='7ebe68dc-4e5b-4fcf-a6de-6bbd0bf988e4', metadata={'chunk_id': '5', 'source': 'jd'}, page_content='이제 에이전트로 바로 찾으세요!기업소개광고문의고객센터이용약관블로그개인정보 처리방침(주)원티드랩대표이사 이복기서울특별시 송파구 올림픽로 300, 롯데월드타워 35층전화번호: 02-539-7118사업자등록번호: 299-86-00021통신판매번호: 2020-서울송파-3147유료직업소개사업등록번호: (국내) 제2020-3230259-14-5-00018호채용서비스 문의원티드스페이스 문의원티드긱스 문의프리온보딩 문의취업지원시스템 문의IR 문의© 2025 Wanted Lab, Inc.'),
 Document(id='fb654c8a-cdfb-4f7a-afaa-3f4afef590cb', metadata={'chunk_id': '0', 'source': 'jd'}, page_content='[임팩티브에이아이] [Deepflow] 데이터 사이언티스트 (Data Scientist) 채용 공고 | 원티드채용이력서교육•이벤트콘텐츠소셜프리랜서더보기회원가입/로그인회원가입기업 서비스1/6임팩티브에이아이∙서울 강남구∙경력 5-12년[Deepflow] 데이터 사이언티스트 (Data Scientist)포지션 상세[임팩티브AI 소개]임팩티브AI는 수요예측에 특화된 AI 솔루션 전문기업입니다. 자체 솔루션 Deepflow 통해 제품 수요예측, 원자재 가격예측, 신제품 성과예측을 수행, 재고관리 및 원자재 구매관리 최적화를 실현합니다.  제조·유통 산업에서 기술력을 인정받으며, 고객사의 성과 창출을 지원하고 있습니다[AI 기반 예측 솔루션, Deepflow]•Deepflow Forecast - 제품의 향후 6~12개월 판매량과 출고량을 예측, 수요예측 기반의 재고관리 최적화•Deepflow Materials - 원자재 가격 변동 예측을 통해 최적의 구매 타이밍 확보•Deepflow NPD -'),
 Doc

## 4. 검색 → LLM에 컨텍스트 전달 → 응답 받기

In [31]:
client = OpenAI()
response = client.chat.completions.create(
    model='gpt-4o-mini', 
    messages=[
        {'role': 'system', 'content': '다음 정보를 기반으로 사용자가 다녔던 회사를 알려주세요.'},
        {'role': 'user', 'content': context + '\n\n질문: ' + query}
    ]
)

print(response.choices[0].message.content)

당신이 다녔던 회사는 "임팩티브에이아이"입니다. 이 회사는 AI 솔루션 전문기업으로, 제품 수요 예측, 원자재 가격 예측 및 신제품 성과 예측 등 다양한 분야에서 기술력을 인정받고 있습니다.
